In [1]:
import pandas as pd
import numpy as np
import os
import requests
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Concatenate yearly song releases

In [3]:
year = [i for i in range(2012, 2022)]

sr = pd.read_csv('song_release/%s.csv' %year[0], index_col=False, names=['title', 'artists'])
sr['year'] = [year[0]] * len(sr)

for i in year[1:]:
    sr_i = pd.read_csv('song_release/%s.csv' %i, index_col=False, names=['title', 'artists'])
    sr_i['year'] = [i] * len(sr_i)
    sr = pd.concat([sr, sr_i], ignore_index=True)

# strip whitespaces
sr['title'] = sr['title'].str.strip()
sr['artists'] = sr['artists'].str.strip()
sr

,title,artists,year
0,(I Called Her) Tennessee,Tim Dugger,2012
1,"10,000 Reasons (Bless the Lord)",Matt Redman,2012
2,100 Proof,Kellie Pickler,2012
3,101,Alicia Keys,2012
4,110%,Jessie Ware,2012
...,...,...,...
20032,You're To Blame,Mammoth WVH,2021
20033,"Young, Black And Beautiful",Chris Pierce,2021
20034,Younger Me,Brothers Osborne,2021
20035,Your Power,Billie Eilish,2021


In [4]:
sr['track_id'] = [np.nan] * len(sr)
sr['release_date'] = [np.nan] * len(sr)
sr['spotify.name'] = [np.nan] * len(sr)
sr['spotify.artists'] = [np.nan] * len(sr)

## Get Spotify track ID

In [5]:
def getArtistNameList(result_artists):
    artists = []
    for a in result_artists:
        artists.append(a['name'])
    return artists

### Direct search by title(cleaned) and artist

In [9]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'


try:
    start_idx = sr.loc[~sr['track_id'].isnull(),:].index.values[-1] + 1
except:
    start_idx = 0
    
for i in range(start_idx, len(sr)):
    # clean title
    title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    artist = sr['artists'][i]
    
    # traverse artists for multi-artist songs
    ##for artist in re.split(', | & | X | x | and ', sr['artists'][i]):
        # direct search by title and artist
    query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(title,
                                                                        artist,
                                                                        sr['year'][i])
    # not specify year
    ##query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

    r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()

    if r['tracks']['items']:
        results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=False)

        for j in range(len(results)):
            artists_i = getArtistNameList(results['artists'][j])

            for a in artists_i:
                # deal with middle name -> match only first and last name
                if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                    print(i, results['id'][j])
                    sr['track_id'][i] = results['id'][j]
                    sr['release_date'][i] = results['album.release_date'][j]
                    sr['spotify.name'][i] = results['name'][j]
                    sr['spotify.artists'][i] = artists_i
                    break
            break
            #break
    

12605 3EV2ZdIDTcZ551RqIRspSw
12607 5OzOQRssjhxaBj0xd78Z2w
12608 2qtoRFCOEL1gRn5q9DJC7F
12609 1K68B2S3aWKYY7coVdcsfO
12610 1KoSVcMX0QEQSvqxTemRnR
12611 5hc71nKsUgtwQ3z52KEKQk
12612 1bPPzeamaxgUf8mwRnjfqO
12613 3n2A6l4UayVe3GGKCvfQWV
12614 0GWD988K3XkxLce5RDIrbW
12615 5EEVWB6SpVG1eRw3qEENsK
12616 7t3q4ghl6rphMXn0oOJhUr
12618 1z15fhSgN6U2k4gqA1Zu4j
12619 6fXhDYxQa1sJLzlfp5Cr9m
12620 6jhx1qxNoNNXRUxwQdUF7c
12621 4BZ8vT12O3No9W0ysKhzPA
12622 4UGWMzkmEPpYoS9myk8lAG
12623 5CG9Ps5ynNjpKJHmwc95pa
12624 6tD8DytjR6btO6kuMDhI3U
12626 0YgXSDMUJlen2AQ6fnYLLr
12627 0NYiG4s8mzAHqLZmJQYfOn
12628 2GBAebLEPMbZ00bFZCA0s6
12629 0lnIJmgcUpEpe4AZACjayW
12630 2puHJmjL4jvO19aEA7kzcb
12631 2NyyzOsGPf1wJF3EuxCJm1
12632 60W7Co2AoP5VVG5Gwu6p5P
12633 620PLI5mPTNMhhdvPKVkfk
12634 0D2jVf87IdcynTmjZpQDrV
12635 6xVcc0NnbFMDwBBzMX61qf
12636 3Wmti6IwvJ2aKeTm9YrZO7
12637 7KXJLrMlgQbGMY8kr6yJlB
12638 3tSwt3KHnIssz3vCjz3FPw
12639 2MTBaxm0ECZuBtfI9KEUZy
12640 2L92bpT2vGWnvsFvnJMvM8
12641 22uBo64kkrceIqJWofM84w
12642 2PEjjPlh

12924 00C5llfInrmXgjBrT40L1J
12925 4tc0KObvco3Evqya22scHv
12926 0DSob5ze3sstCI5FFpCRwj
12927 70DpnF54lwTlUsxXNN874H
12928 7h0Io7i0y7ojdqbBA2B9g7
12929 1yvMUkIOTeUNtNWlWRgANS
12930 3E5dgNJBahIRdJIzDqqYAz
12939 2Y1LWNrO3Ls6tK9LZVehXk
12940 2ImWMn8Pqxa8gHfk1ZJvAh
12941 4pjiK3ZbfKcRt71VzgeDOk
12942 3V8sM5OOG6YfDuDLa2IIYJ
12943 6OKOwqVfeZ8cb6bicdjkzT
12944 05m1GXCzSikr5HBveHXKTf
12945 0kN8xEmgMW9mh7UmDYHlJP
12946 1T7FpBaAFM5aVXaJZ6iPEy
12947 4hivfNEXusPBa6DuCpdjfZ
12948 0qMZ2EgknkjErV4Fipwwdx
12949 4Qtw0HNKSbIRT5sWUnbRXc
12950 6TFGNgCyhgHKNn046iG6fa
12951 7MjSipTto9QljYzZnloXOn
12952 2z3joGXrdGL2iEjIKQTiN1
12953 3krSd1p6qKsmzx7Xm1bLEW
12954 27oxSfW0iib7w0NG4zf9Fp
12955 135SEcW19xWXM3WghwdUBa
12956 37kiXtTUprX3HvjGvyTDb3
12957 77dRMsf3qPiG9DRFZfRg3p
12958 5hyq3LBlCfjRQAFkdQwe8o
12959 36t2fiZophrDtYBVkrlCQz
12960 0HLoOLMfvYJpMba3gC4Z3R
12961 6JNS8rIcTvKVdgbSuQC9Xe
12962 0Rxgdpe1zgZ6ZzHgxSaYEJ
12963 6BOtDFg6GJm0rPUDuNmx57
12964 4ejlJw3S5NgIVgDI1izxMp
12965 4Pa5XXpn3sdBPMVCSWGEhB
12966 7bPLo5Fv

13258 7y9iMe8SOB6z3NoHE2OfXl
13259 1vqfKzWKCabKdsyuHiWyvx
13260 48jPP29QyeTorwlkflGast
13261 3GOjD8hwXOxbTF3apTqSqX
13262 2fZQIJew3nkNe99s2PKzul
13263 63fj7dR4QGRHDHKZStUuAY
13264 0LwbO2KIQw1uR2FCqwaKAu
13265 5oBy1HrbZ57X8knBnxBUyC
13266 1ENSkfM1Lnyqo9auiJAt4H
13267 6Iuh3vNjpWhSeHM3RZgzQL
13268 2fQrGHiQOvpL9UgPvtYy6G
13269 3Lfiu5sZ4M4B6JaKMBc0FU
13270 2U5cq89GCnsR1yixKkC8d5
13272 318EGw5ZZB8U5MdMWQwxZQ
13273 5jwOPdDOrVecEc6rCwBc9I
13274 0YtOYCIGWQZMDzfBL0eLWh
13275 7Bmw1zhoY4BgidZrBNb6tG
13276 3csEQ8iV47qDzfJysPJbpP
13277 7mFNmurtskRmkxR01mYPfx
13278 2MW0ofGJTi9RfoCMPsfGrJ
13279 5R7TAOPx2H0ImjCkdb4PXR
13281 279Tu4mcAOwEMjybwpNhXR
13282 0lUdYoSr8Hm3GL0HZld4ac
13283 3Ue3s4Mj9zIvWzey7zUmQT
13284 3O7p9Itz8PXUoAjD2vmuM6
13286 1P6JClfDRfWlDTYUhphWYQ
13287 5EFQmntseMD5F4FkwLoND0
13288 1OG4nxGMy72EklX2DwcN3Z
13289 7iIWHoKyYdDVYSMhivbCNp
13290 4pFllfWCcQo2Odvi1q2Qpt
13291 0eWkGHUqc5a19PIzJ1v6E4
13292 55o72A8eQo8UVYHbKf9aXX
13293 0pqnGHJpmpxLKifKRmU6WP
13294 51NFxnQvaosfDDutk0tams
13295 2I88NEWp

13569 65ACnVMmePTp8Xdk11jP2y
13571 1pJQAHpD51J7GYaFrrFO9S
13572 4OAGbsFAlS4bpcn056fpRX
13573 3ySU5vwQB33iGulwcUL9qQ
13574 4JNdKmSZRsYDBwhIgZpa5M
13575 2ClpbVJnjvYLpYqYhSnEkL
13576 70PPbwGrPBpQ1hRBuuSirh
13577 4hLvUbH5dkCByoNFsM02Js
13578 1K3BOKLjzLgO2olPlWMnkF
13579 6Gcv1c96EIihHdJeERkpc8
13580 593W4qZOwwdqF6YnimJjL6
13581 0XQHz5C8PknyrEhyrKxVKa
13582 1Vvr5Lf5tAbPtgVWA2HILp
13583 4GkkxswcdEt1tkSyiRzgnC
13584 67l1tytshju3Ily8uIAVKH
13585 49iNhnDmNnHYxPvEIVtPiN
13586 6oVxXO5oQ4pTpO8RSnkzvv
13587 06u5LrUpbosQlQ1QJFhPpG
13588 1mJFIsUzfbeh0KBuXLVe16
13589 6myp8eBOKdeaJm2g8YsOZz
13590 3FZuZot3AXQVXaTlPWpzCc
13591 7srSfTe1zrNuUY3rdS3ez8
13592 39u2PsqrAZpZ0KPx6CjasV
13594 6CtJlacMtspgBVTzZG5aCM
13595 1kFBLqkTuNKlspkJ1EOGjN
13596 0y9uTzK9cNKSAEHnpeRG8C
13597 1j4kHkkpqZRBwE0A4CN4Yv
13598 6ZLHZYOVrFzUMtHdpdFTJ2
13599 483NHOdp19Ig0cRkMFUZtM
13600 7N0DD7yd9mXRnvmJMYR9Su
13601 124qbCWKsFvtyzbu8b4HJN
13602 6NfyR4V70w4aoZxGVoPp7c
13603 5jQTcDXjhwYjyofy6SBaoW
13604 3bbd4xN5xjSfIJyoyWZMkb
13605 7poxOjmt

13889 32zF5eXrb8QrUu5qNdYmIO
13890 7gkiBjpe8iHs5esLLlThly
13891 0OI7AFifLSoGzpb8bdBLLV
13892 56WUQRA9ylzIKOjKgRPYM0
13893 5u64RlrnvvbtNh6R0EMh6o
13894 5E5MqaS6eOsbaJibl3YeMZ
13895 01ryv8tqj6gZnGb18srxk7
13896 6KjbNLbRjuoa8rEq5yNA6H
13898 5QTdOvIF2ehBMZpSIIGzIo
13900 3DV49ruvI5Sl6iisPZAc2c
13901 7r5X8KL8y0CpZZErtmOmNo
13902 4rxll3CP6XklLuoki7A26P
13904 3BymzRCyTpah2tLfjXuMok
13905 5nHOrFZM96uJHWqt9h2nZ1
13906 48yvsxGXnJBxy8nXgGbs4S
13907 33471tzU1VghDg9zNgGRpz
13908 4c75xeSteTcgSSxRtsKqOJ
13909 51txLjbvSBRvtxjCjAFLkM
13910 6AH3IbS61PiabZYKVBqKAk
13911 3ETW8l9FuVFpPHUSTyZDK4
13913 2cmWjJEB9PydEc1eIHbOHX
13914 2woOqowlZMgYyvGslyBDtt
13915 6PJ8FF6UR8FZXfEvpHkIVN
13916 1bm4WzPddQzOZFAz7JTklq
13918 5HPT6H8sKOhkGZ7TCrTgfY
13919 4Hp1edCb3VO1kghZ3L5qtL
13920 7KXjTSCq5nL1LoYtL7XAwS
13921 5HJ2psjlq5wyxtUA43Ylgy
13922 6xHI9KjUjYT0FPtGO8Mxa1
13923 5LN736t27w3aH0lSAMX4nD
13924 2EUx8gP2oWNM9olCp1E2M0
13925 76IkF7pA7RscjDTvOFT4vb
13926 72KDkzE12InC0ahlMc5vqj
13927 3Qp4cWoZmfdjAx5dJFSHyS
13928 1JuChOEP

...

18586 1kBx9VGumfuvlfqdlAGorE
18587 5MwFstlyAOgFPOOTgzPjjg
18588 7qRVyZDt1Caftkk8CQlKtk
18589 0klnCzWGjoRQBjoPYe44Gl
18590 6oARVUWXiWceOYeijMwcvw
18591 4G0WjKFAlhVSwmulSYzPsV
18592 1ViT3UtxatnTKvzv7VXhm2
18594 6xGzozxqIkO1ofWerpXQC3
18595 1CCU4UhgjeNIsSZolR66VV
18596 113vSUnh7fsCbSRNgEH0to
18597 7Bar1kLTmsRmH6FCKKMEyU
18598 6MWoRt97mnSTXZhu3ggi9C
18599 2yySWxfeRN2QRZnFMnYMQl
18600 5428XgSLRJfpMRN3r6YXob
18601 6zFMeegAMYQo0mt8rXtrli
18602 4oqTFpt6KaGFYc1YHtl7GW
18603 5u1n1kITHCxxp8twBcZxWy
18604 7xMFk2Vj2hALyMdzHFopJl
18605 6vxU64pcIuQ38O015Zy4qj
18606 1s6ksEc1H2lWjf1HIcwq6e
18607 3QnDNa8zN8iSXPn3VDC471
18608 3lay6f81Lhy0qp2Uvp4cNJ
18609 7upm07Y3Fx4AJs5G5OqG1A
18611 5rnqbmSmvqKzDsGZoqeuTN
18612 03WIqWZZSHnzJF3D18x8Gh
18613 0BU54O7NqOY46oaeRVnK7A
18614 3qwPYg8xTWPafQD3YUNaTU
18615 7ydjUPZFQVYZ54NLmoISfG
18616 4SFknyjLcyTLJFPKD2m96o
18617 1I45GWyejiphTaj0UJPkEm
18618 6ofMKJjMpfDEb48JYEAgjX
18619 4c2FRwG8WCRqDrY0hPgJaI
18620 2oWuBmQtYMx3LEgahIsJvs
18621 3R8CyhJfVjvgIROd5RSGhQ
18622 4TfPtTbJ

18899 5WDIq9E5f5ryHeIs8FwKcl
18900 5XqDJFVCyRTm5J7cIfRmR1
18901 5rMJJPGNeRzouxRbeo7src
18902 2F20djOXWDSFEXu3tPdV71
18903 1af33AUaAuPMkXuOgD30T5
18904 3Q4gttWQ6hxqWOa3tHoTNi
18905 77hitAMfc6USq4gkpwthV8
18906 5ciyqWW5YuGqF4Avu7uwry
18909 733c1CWmIGymoQXdp7Us88
18910 6MD6xpFK4cfquxRqXxqwjq
18911 0q85C1JiECyYIQthz1posA
18912 1BLOVHYYlH4JUHQGcpt75R
18913 7E1jVNoWuemqUryI4FxsVD
18914 4LKH9EGWayUvhM3CzmPVEI
18915 3QiNQFiueqnIc6rOPLZB3f
18916 0h6vLT1rLneoc5I4sOxvB1
18918 2FX88wSbz72eeCm9EGIZqN
18919 58qO4dGa5SgNdtvqvpewyz
18920 249gnXrbfmV8NG6jTEMSwD
18921 1jkRCIT4lbw927QNmt0NGh
18922 4Iedi94TIaB2GGb1nMB68v
18924 1aKQlttljhiHZtaMXqpnUi
18925 26G5nmIx7Yvq79bIIluyZV
18926 20Kau1BL5IUaPo4l488D57
18927 6NhS5LwYbJ6xD7BGvlWRJO
18928 4VpkCmCnqgcUcysgHdTbYf
18929 5ozqshq2dtU7SYCpCBu0NE
18930 14GwnOeC9qYEKEA6uOZepa
18931 3RlnNTs9Cot4M1qL2C9gi4
18932 5NUXE8W12lWcUXgJRCjeEw
18933 6BzU6BMqMd4qelkWTjLwKy
18934 25tvfN4vl4jtYYPYEi7dFk
18935 2slqvGLwzZZYsT4K4Y1GBC
18936 6bEZIqFksyE4HGBYLUGAcT
18937 4OENnoid

19210 0UZF6IktdKFrhDu3eECrxM
19211 0VSzREd1OjEWJ9tXoFHRQH
19213 3FR2w9O5i9q3unY3gyjq6E
19214 5DmQnsW4ubHlAKK1IwmMA8
19215 7bMYtlICTx9mExbnIBeeIV
19216 27u7t9d7ZQoyjsCROHuZJ3
19217 6rIIhxmidzI07z1kgiOqY8
19219 5H4mXWKcicuLKDn4Jy0sK7
19220 4mOhgByssJQ5AdFaliWyEc
19221 2TfsiujXvoafnmkVt7ZVnL
19222 2jLhqJAX6TyeUuaR4hIAm9
19223 1YM6c2sxbf2Jacl9IFgn4I
19225 4YAg8h8WliHZlW5wuv8Jpk
19226 2ZTYlnhhV1UAReg7wIGolx
19227 109QdOfCgaBYIql2aaYg2E
19228 2Cj5LxM6O2HWQIr9p9h3TH
19229 0PurA4JVJ8YQgSVopY8fn6
19230 5QZKcQDp0tCcvMdyqEciBV
19231 666PdikRnUzdvjT9p5wWND
19232 6gX7cEbJHzohlIXQXL7Pkq
19233 51CeiXQgEq3YD4D6v3GjfB
19234 7DmTaJoM7L020qm3egqNsM
19235 6teqwuB74aLYjj7Ahuz5Hx
19236 127QTOFJsJQp5LbJbu3A1y
19237 2IHroOC3rcvR8fx0SN8Xva
19238 5nexAvWNhwg51EavnDgViy
19239 5knK9SgWn33kQPCIPgagum
19240 740kQ8xwSGloumdiKR5Cwz
19241 3dVvWnj4D8JGkKvo6Hucso
19243 0CbDc6JeCKQGYVAI89abdm
19244 1RorYCwbxKziBZ1c1bbXcU
19245 35Q6E3BnFbU57craZaDF7O
19246 0V3M661h6XPlFD3QiFkbRt
19247 1E4wqt5DjnD1hP7gGR5KeJ
19248 6wQD4iIS

19521 39nc7C7C7iIJtVAlNznst7
19522 2NnJpRXIlx35Vij3bPZO0h
19523 1v3lhrZEsOcazqR2QIyyRu
19524 6ISZ6sKcNBdYJS7XYHm5cV
19525 6HU7h9RYOaPRFeh0R3UeAr
19526 3DMs7bahbQTnoxCjgP7qoK
19527 3kNWyHdLVW1x6pn9EnSQ1H
19528 2gT72HL4Y84K2Mle3miAMJ
19529 11C2O5zYbJAxtlb1bRPfqk
19530 7bREivF1RTm0vOYB2wi7nE
19532 0zKvnr0cLQIrlWkqInTRJS
19534 1058fW9H3fZA6QjYCdOBad
19535 2ixabSVbccs9np9r5CpbWW
19536 4I5bvu2KDsrCg0EWHIcvul
19537 2AHnmlkyZbnxqSA66B7jK3
19538 45PF1Y3RcW5MK0jxWvhc4D
19540 5wANPM4fQCJwkGd4rN57mH
19541 0QULNNd9z5s35entfiiXoa
19542 2Ms8MDDtYBw7M3K5F3EqPX
19543 7N1TIml2jRQRgXUrpXgXrk
19545 6sRmaXQZo0EK2woVn2COxM
19546 7bgAiDFWjoPt9cLscanvGv
19547 5Dmq7XPXXcATuJnSOj8UrT
19549 2TOzTqQXNmR2zDJXihjZ2e
19550 4bRPecD9suGW8CfNZhzjS1
19551 6jnDRGCEEb9i1M37kHJJSd
19552 1EJIcDYXwSqipW5dFe4uJz
19553 5jhBwnqzNNrENXnYrAdoCe
19554 5oKBOiiJceLlAEQ4VaVe8p
19555 1cm91sh5RtWJ7FCCSPm5ei
19556 7Gw9DRZQNuephpqDLkRMnj
19557 4KsrNG85gkH4u3daIfZ8w2
19558 40iJIUlhi6renaREYGeIDS
19559 7Bpx2vsWfQFBACRz4h3IqH
19560 3ZozFqJJ

19828 0XDlwVNtd6wvp3PGGsghqf
19829 7eX3um6NpOQKWJMGCi97XD
19830 23ATJFV9URwzhv0aYKzJYp
19831 5K1XaSkO5TwfGjTXyypc9Y
19832 1Qsl7IdLvS32YKHll5b15k
19833 00R3xLkJsIxENOc3qA4CDA
19834 5h0DGzgQpKTyXL0bH5cvXZ
19835 4laAKIq9ZxBCwf99rauPYb
19836 1aU1wpYBSpP0M6IiihY5Ue
19837 0mehSdTadpXbHAWTrnrIXO
19838 5i4WgW0TL9VSosy73Q3Ttv
19839 3zwMVvkBe2qIKDObWgXw4N
19840 302p1fXtdjYhaOEawMwNyr
19841 4Agtk2MrapdZAVN7v6PuFO
19842 0uuqi1dTEf5KJDlFsKWHKt
19843 6KgtcmCF9Ky68XC7ezxl3s
19844 2UbVnbE5FH6008mAm6Mmgw
19845 3PeCWDczrGqdD1E3mbok2X
19847 2VmccNZz4zFmlcXstY8a2D
19848 0uRrG2jRR5tuifsYIJHEao
19849 60dwJ8dnqZ4WrIdiu7LkNR
19850 0EredcGoVUJTRiBLmwI0AO
19851 1TsiNyKjoht5fcttAmlD4J
19852 62sXIT6kAdOgcWqnhEZaRL
19853 2qg7U5mbHDFTXNKvPNF0IZ
19854 1TZv3bujNaYz646eezRE91
19855 1H8U9f2oVZpb4tmR7yO3ZQ
19857 6BzgLqzIHmRFkWcNJKkThS
19858 0exZ0YogJPbjGzblpcZaw7
19859 1q2Zb0v5ZGxKDktEz7wSPb
19860 4z86ZEcMLofosRyJMqBkIz
19862 6ggoPGUGRzXBBArldt4ESv
19863 3J4VKkUpLvomXJbn9ZAypO
19864 75MNhvTCCKsST3YqqUiU9r
19866 3MthJpM1

### Traverse artists for multi-artist songs

In [10]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
    
    
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    # clean title
    title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | X | x | and ', sr['artists'][i]):
        # direct search by title and artist
        query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(title,
                                                                            artist,
                                                                            sr['year'][i])
        # not specify year
        ##query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()

        if r['tracks']['items']:
            results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=False)

            for j in range(len(results)):
                artists_i = getArtistNameList(results['artists'][j])

                for a in artists_i:
                    # deal with middle name -> match only first and last name
                    if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                    ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                        print(i, results['id'][j])
                        sr['track_id'][i] = results['id'][j]
                        sr['release_date'][i] = results['album.release_date'][j]
                        sr['spotify.name'][i] = results['name'][j]
                        sr['spotify.artists'][i] = artists_i
                        break
                break
            break
    

449 5j0SvoSBSzBaXGD0IQdmA4
1071 2dERV62dG0g16ACbb7P2S3
1247 3BzK0pce5dIZHpMXR3VPJ7
1290 6urbrGY1nxABJj4tqz6FIH
3248 2yi7HZrBOC4bMUSTcs4VK6
3319 68PD93zd4LzpxMWDxs5ugh
3755 7n4JJDiGCGoc01cn439ivX
4812 4ws6ARKJGwPHWOXucBpiDl
5012 49zAFEviq7WrEGWuYL2CtN
5387 3sib8YIkqyxKokUZ7zy0kj
5460 48Xp0fGOluD1NJmHOeYFg5
6371 483XiZ5o13Cc1zoWV7jGml
6562 5xO5ON84ieWrb8wYz7j1v1
6615 3YB9cvd668HXBEq8rbBW8P
6663 2ELVVIbpucfOqGFC21Q4yR
6688 3GKDM1IEh0GWFMR7g5Lexo
6788 5wnfZ7VjDpZNyfJkqc9I1d
7606 7J0UbeV4G3iJISrFm1aWN7
7631 4LdKhiumggIeOPIVKOJ6eQ
8055 0FTzAmvMu2zKdp3ZrvlliF
8440 1DmnEYXa4WfbdhAPwNzgD8
8668 7jslhIiELQkgW9IHeYNOWE
8715 32k4TVAGwX2qQzdrkhRgYi
8961 2AGottAzfC8bHzF7kEJ3Wa
8968 0GR7iJLhj80KD5LkA14ZRn
9033 1IecEbWHvnC6kiwDutTUX1
9247 2QJnTfMpNG05KFf2E3gVIJ
9586 2ACys0pX0SjmpQmQWzp7wt
9601 27GmP9AWRs744SzKcpJsTZ
9724 2uTG00P1DkhB97pdITkl2O
9820 0tG61juk79S6dEQPVgwGgN
9858 0UIk2B5sfUKMBAkAyLZBiB
10116 76kyKtPLsFbQkdQ86QrkF4
10241 2bNxCVI4Fub8BAwr0CQDyd
10861 786tePtnXq0yIi3gm0xWhD
10925 2j2Mz2sbBWw1

### Search by title with artist traversal

In [11]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
    
    
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    # clean title
    #title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | X | x | and ', sr['artists'][i]):
        # direct search by title and artist
        query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(sr['title'][i],
                                                                            artist,
                                                                            sr['year'][i])
        # not specify year
        ##query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=False)

            for j in range(len(results)):
                artists_i = getArtistNameList(results['artists'][j])

                for a in artists_i:
                    # deal with middle name -> match only first and last name
                    if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                    ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                        print(i, results['id'][j])
                        sr['track_id'][i] = results['id'][j]
                        sr['release_date'][i] = results['album.release_date'][j]
                        sr['spotify.name'][i] = results['name'][j]
                        sr['spotify.artists'][i] = artists_i
                        break
                break
            break
        except:
            pass

1 2MobJagQB0dGJoRqI8OT0m
25 2WKCH4ISejDV9ad7iPp5XU
26 4Cd1nnR2RKdbFN2eTaxAFR
170 5ikG3xm47oLXYTmkun7uQy
232 24m68kElKhXrlVUuBFtqaY
270 712uvW1Vezq8WpQi38v2L9
411 6Be0UjOGptTtj7YWBUKIs2
424 6by8ib53OiCTb3zDOUV3Wm
465 7BhzUUgco167ieqYbmDA44
791 2lUK4wPSPAY5pwUEl30j5Z
835 5OVjZ7Cy7U2gVJX6eLqmFT
836 3zElgyWyRSSe3XHNcsGCjO
955 1M22aV69KFPVXTdpedurxW
1037 12uxpGafkVpNeJ4MWFRBin
1092 5DSUE454oiPSIQUgQIhCDp
1094 7giOLnVogBHlKnQk259rtv
1338 7suNOOu3xrgMxDmQxAWGCv
1405 1jZVdhnvev7YX67XiwKNgc
1406 439TlnnznSiBbQbgXiBqAd
1754 3oh1i9GfuAfy6h7ezc5OG0
1755 2i6xxOLeUdrnpJOnoXDp2b
1785 5AlKSrMjBppYiBcGAE6Xe0
1849 4xNH6mCQMSU9RzE60lKbul
1886 0koKTX8V5Gg6qGrZX0x9wo
1913 5K9orIcQPHZwLpDFymqeYh
1917 0qGFnxDIqGBIq2D3zNbjk9
2340 471JtpRQ0oox9OoZcbB8OO
2579 5i559orX0oL7i4AuZGKott
2630 11mayNcPh4CxPfvIopMgAo
2647 4QoFwqgLw1CtoeSv6N80mU
2659 2b3LBsDr9ZvMIiy7zxEjj9
2704 1DdYlujfj7sUd6HxJF1Pa0
2755 3EaogCkhZTnvP7zYmRR86P
2848 4lbdubTAn1l8XZc4803AJf
3018 27BAPGLO89JfT0Vnl4D5Xh
3163 7giONDpA2lgyQbmfrln2qv
3221 1XTp

19666 1kdWw77ZpYOkhxeuhzU1j6
19680 0MWiSBKm8Avs8iDIxcertp
19727 1t0Jmqg1pKVBbxjQFZebeR
19769 3TeOBBmcUji6oW5RcOybUA
19794 0CfPMZ1DSdzglsMRpLDM2H
19804 6jy9yJfgCsMHdu2Oz4BGKX
19846 6xNz3BbD82U34ogqxVpw1Y
19861 0rGanUWxLUWXqAe5r0pigY
19865 6ihxXxSjslaVYd3oaAHCMT
19885 0EYDsKvLtGMdvsA7PaUvJO
19915 7eResoqEJJAVTkQYSqvO3P
19938 77tIzUnEdjigwTVJskepSE
19942 5zNW5ARjYhJQJz1AChP33n
20021 1PDP7mLiAMwhfmgIwzhOm2
20032 4IpnOxXQPZgPLeJ67o8CSJ


### Search without specifying year

In [22]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
    
    
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    # clean title
    title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | X | x | and ', sr['artists'][i]):
        # direct search by title and artist
        #query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(title,
                                                                            #artist,
                                                                            #sr['year'][i])
        # not specify year
        query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=False)

            for j in range(len(results)):
                artists_i = getArtistNameList(results['artists'][j])

                for a in artists_i:
                    # deal with middle name -> match only first and last name
                    if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                    ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                        print(i, results['id'][j])
                        sr['track_id'][i] = results['id'][j]
                        sr['release_date'][i] = results['album.release_date'][j]
                        sr['spotify.name'][i] = results['name'][j]
                        sr['spotify.artists'][i] = artists_i
                        break
                break
            break
        except:
            pass

12 5st6ogCAEzsPbqOJfoJfbT
41 2zW1XlHAazTViSVM60yh6z
51 3IJGQEgU7pZOj1uDHIPfwY
55 6PDDLs0XDsWyhJxIqsgzWW
61 4sT8kcliEkMJBUTQqKOIUb
82 2FOLVyfKFWzS0F5WzC7NLD
86 5QrqXOTpWw4MfoDlHfCdBL
88 0USZx50eApN045zIIyjePN
112 6C1BUMo3Eq3Xudv2Oo2CLr
134 3Rh3zDYanZFHaeXFT4Asps
140 2KndZ38hCNVk3Jmb0l6uDC
168 09XGGn9lYHT8Mcsu1oiOJP
175 7LrgiRjCm2oKcuJGbmk0pg
184 2HHiH52OPxe8qyVO7N4uH7
204 735SArN9pqYYzYg0leQ8nm
218 64y85LeHY8Z6OlTOM9cpKD
219 3bwCMbwDZVtvJDnUTQIdCX
220 2EUHzmBZux7SSra71cGkIB
258 3DwN8YfAtvJh4c5u8DZPen
266 0NoKPhyT2DZ8yxtVvT0Bad
311 4IFTlLCrZsToaJ3cYlRdpb
314 71fbIsRGkm6WG9CTGyvEum
358 78GjajEEFelJf4Ibk3q7GL
361 6mEDMe0zG5pYrsRxizjauk
362 0zmMSFU9c46I2mmw34a4XF
366 1rIvIifhYqdH5OHQjvbykW
367 22u0bVblodXzifcToTjrC1
386 2Fvvez4jjuTF5vrDESR2qV
408 331q5cdwo9aq7ih6bSpoOa
421 023OVLNzXhX0j7CxswUt6D
428 3PNYZeKsUfm1ks7giH1Myx
433 6plT7nFGiXKSBP9HFSI4ef
438 5cj2g0s91RSoKpkAyqs2gQ
469 2kWowW0k4oFymhkr7LmvzO
482 6I5cQVYvoc5dqpyYvYHpSw
487 1SeZHSmP09eYHXfMoGNrLM
502 3mIinBcQwppTzuPPcNNRhz
514 6mH3q

3924 2xUHiyGeGrWmmfv7NbDTWC
3931 27ZhKzjHoE0UUlyMXu0ZFa
3935 2jNSYcYDifZJAAi5gMVJR5
3948 2Ih0U9yUuY0oU2In3h21If
3963 3JMyRsMQIeRXSWwFbO5gLn
3982 0ciA43AxH3rZASCUBmYvPJ
3988 7lG4154Md1Kw7BMg56Pt4s
4029 5OFnCw227Q1wr8cEEftzv8
4035 3IYn7otWjOG24CDsEVKyXi
4066 20mm07wltfpLJ6vKbx01da
4072 6yjsa6c8nSGQqXF12vFSpp
4084 2kleQWygkA5yDpt4zIfWzd
4102 6IDDwI0YOCAUDhMZltQekS
4113 5IXV375WDokSNjZfR4GgnU
4114 2RAXAF7qjvn0FsRuMMTlBx
4122 2MDueMvPkUTpU6Xkmeinln
4123 1aQuEPiDbYD0n3ob8YKBQn
4158 5SsR3wtCOafDmZgvIdRhSm
4194 3h7pI8oM2CpXiNSZbQuEtV
4197 63FrXif0Pdu4NAPvTh87mw
4221 2Xaw1RrXP3mdooKWysuRJ8
4234 6np7WP2rTg4ofHFyRWCM0G
4239 5XEZx88UTiwb01Hg4GRi6H
4264 6UmZNpLWzkRWeln4KG0K84
4304 4DActPOAtak2m8meZeMt3B
4355 5Amm6v8Af2mkVzoso3XZcj
4382 3TY6q0ZeSVg9byfug6KKrU
4405 6wQswh6JNND7WwelaYyNhJ
4409 5K8cyMKcnG4rx5ocW3u7OW
4416 274EG72CmKSa3mWBsacoL7
4419 2nIulsLMiP2SMTDrtxfQXX
4438 0Yp3wCUpjrG7NWWPDpmQXm
4442 2FRNaqLU6UFv9kANvg8D7c
4443 0nfUOSvcUfntw9a44vtlOb
4463 0fAJ7GyI7NHDD3YhuNgdDP
4495 4XOVGSQjmxtSbt3

7719 7m88eIVGz5yWoikkHnoVIf
7733 3OqwM9CRqLskni9acTwuj1
7739 7fLHguF12laXV09NVTrlSD
7741 3TmyCY31Q5MEDV1nWWrYAR
7755 4eQznxonJdR6NEksFNHMCT
7758 0rZF3zZKA2oVDiwW46WL6l
7761 5C4NMi4fPpdFPnMZVsyG8H
7765 0ITvLv9ys64WXJjJ8pLkJi
7794 7J0mZi4vIRqBR9pjAt90nX
7807 3uF2FEJ9Y8KYnSvb0B1kWS
7820 0cUYHAciIIom2n04j9p5qj
7830 1IDF4jYiuI17U4BW23EoXV
7840 4uuddHEP8cr9EMAVkSNI9N
7855 7rnhxvGrHEEu1KFzb7fpxh
7889 1fBvk6KaOsuUfXaKU1M4M9
7896 5teGX9dkVCZOuao7dOWs7l
7897 39J9gLwAyM6n2c2vFVVvdV
7926 2U9kDk5mlHYunC7PvbZ8KX
7929 6ODoMQe0JwO6bgGV1JqAUC
7957 1OWv06xSpcaNE1eQwUJPvF
7965 1mAPo7CPlVL3HDT1ZRqhLr
7966 0hjANar01OL1RWox3nYT7k
7981 7F5J5cfDnF0h9s69zoeV4O
7982 5oNlBs0y1lB8tBA65E7abl
8004 5XWqdj5nPNVeM2TxIiyY5W
8025 4eGVm6LoWxE0oyVGAmgNAw
8043 3Mi8mCnvcBPdiPgmHYWEow
8048 0KUQVPAXbCrvVrWslGZXSh
8058 0588kHJ34DF7ZvTyuo4Dnk
8083 5uqphUYsR45kPYplrYSAQh
8100 0v60rM2HLSatgUyVZDrLwm
8101 0k5hoseEJnCAbpRh38dNoI
8105 053s9IvqFRoOi3Ud1epOD4
8107 68m2FTTwwBc3a88b9w3UOl
8115 0nrwXo4inMMLcV3ElQitxG
8121 3SVsgASqZaDDTtL

11803 3YVg4yurS4YaExuUDtSoYp
11833 12EKNVYavyUOF2AGeiisNl
11843 5C6uh95eAL0RBTSGXKQwvk
11847 35lBL83KZl80dHivikZKEq
11849 0wb0X7n3cAKeajIOguyd9t
11859 0ZUCKcWdFABozlSmojDyLZ
11872 3V7WIBmAFOpu2QIrkYoUeP
11878 09iyGiljLWRYXdRazFkdtJ
11889 2jB9QaPJyVdz2Q0Va8rrnU
11897 1wIDxpXCRnHmQJOvSsVIP1
11915 32jNxnHEdW0R152i2MFLIC
11920 1MAtajbUK7QNlvMIlDAxMz
11923 4VA9tLqZLmQ3HBxO5I992Z
11925 5NbBEzfLZc6anPJYlxNpG1
11935 4Az17HfqonKSWNsaLgTBeK
11968 1Cz73d6PqsQwSxXhfLrPOk
11998 3sdiDU9CIyic6Ts2HO2ucD
11999 3wQjHSfCaPyPatvz9sn4w4
12006 2f4FDuRjuV1yXfpoG49ABM
12021 5HqpTLBs9TbMPlUb5MMO60
12025 15Lw8Uofm03raFJeuVrQ0k
12029 5dViLbqgaPvvLN6IS3F7HM
12032 3rsOJ5Nbmr7pWxY1p6ZDxk
12049 6r17TK9vMMuwTTL2p7O3Ac
12064 02hx6RMAfqOVdG3C0uyTFg
12072 6eUClMzq8c9cKBXd6NrgVN
12089 0ceXGMDXYHcRIYRPnuuY0u
12096 152lZdxL1OR0ZMW6KquMif
12100 5hIw8IeZ1UcxB8yZrEy9rc
12102 2Wj1AZdBPMA1mkuuoTfOPg
12133 3YM0vEH7taFj7OLW3dWfiu
12156 6XBkL7qS5NuSB8bvyJF5Kq
12170 3Qn6PLKldpqJMFK02PoKdc
12185 5e9aFSdnkQUrm5y8Dhw0qN
12199 3PYZWh5K

17455 1iyE0UwRtLUBf9OqLfFGSp
17457 7eOkHIcaoUhngxxypM5Ajr
17494 6NrWdxWR0sj4gPAOU2w2tu
17558 6jGyDvL1Rf4XfwTb25o1HN
17577 4drTFbY9KJIvllrqVcJvLi
17610 5T8OICEqdqbx8Uz7L5abkH
17630 2ce6x2fCzlgI2fnCAGJDUu
17644 47qZzTLLJcuD69AIKnquo4
17652 2cytBOLpwFRX7J9URCrFIe
17694 1KcOU0q9AMXV9xi4tSzzCc
17695 3hImfCYVAHQBtONi06hqs8
17746 2VPmBOuy7ZAOFSzKwW2IEt
17770 1DVWkJEjgVStgU2EbR9vpz
17846 0o1xsthxqPUiKqVqRN0r56
17876 5pSRBmJhkfZm1eilG3VW3L
17899 29aVsOfrhR9chb2TzKW2dh
17917 0LybHog3IZ7ywvkgkXrBW7
17939 6BK8BcFxMFr3JVUaAOZsYC
17964 15C4TnrrVdym7UykxQIOTZ
17982 5mcxeKzIS8aDbz335w7Bq0
18061 6QZP7wxsor0vZsI9dkKZk2
18082 5avVDmK9FVJpd3MDO8I3Zl
18223 2kFHjWko1il6O9L3eK9IzG
18224 1fckqKAI9ug7U1DgQrrOop
18338 6MGRweoLPBAaPcXTMpMWsA
18419 4jkwu5KQbXXrbAZzVmt5fE
18466 2aq7ZO1is1czoQAh076QFU
18508 645Exr2lJIO45Guht3qyIa
18548 205fvU0chRUNVGLu71zD6B
18565 6LVzOLJkzG3C4hLNOdaBry
18610 3Gyl1BvC41QD8rXEMfSQWY
18642 6xzywdfai33n1cPKYF0iLW
18679 7k6OXwl5LPbJXZyQOPWSa6
18684 0MWxVd7Hk952psJoL249TT
18691 0B0Kv1dO

### Search title with special format

In [17]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
    
    
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    # clean title
    #title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | X | x | and ', sr['artists'][i]):
        # direct search by title and artist
        query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(title,
                                                                            artist,
                                                                            sr['year'][i])
        # not specify year
        ##query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=False)

            for j in range(len(results)):
                artists_i = getArtistNameList(results['artists'][j])

                for a in artists_i:
                    # deal with middle name -> match only first and last name
                    if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                    ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                        print(i, results['id'][j])
                        sr['track_id'][i] = results['id'][j]
                        sr['release_date'][i] = results['album.release_date'][j]
                        sr['spotify.name'][i] = results['name'][j]
                        sr['spotify.artists'][i] = artists_i
                        break
                break
            break
        except:
            pass

In [84]:
#sr.to_csv('new_spotify_correct_5.csv', index=False)
#sr.to_excel('new_spotify_correct_5.xlsx', index=False)

### Check title differences

In [32]:
'''title_dif_idx = []
for i in sr.loc[~sr['track_id'].isnull()].index.values:
    if sr['title'][i].lower() not in sr['spotify.name'][i].lower():
        title_dif_idx.append(i)

title_dif = sr.iloc[title_dif_idx]
#title_dif.to_excel('new_spotify_correct_title_dif.xlsx')

new_title_dif = pd.read_excel('title_dif.xlsx')
sr.iloc[new_title_dif['Unnamed: 0'].values, [3,4,5,6]] = new_title_dif.iloc[:,[4,5,6,7]].values'''

### Check duplicate track IDs

In [41]:
'''dup = sr[sr.duplicated(['track_id'],keep=False)]
dup = dup.loc[~dup['track_id'].isnull()]
#dup.to_excel('new_spotify_correct_dup.xlsx')

new_dup = pd.read_excel('new_spotify_correct_dup2.xlsx')
sr.iloc[new_dup['Unnamed: 0'].values, [3,4,5,6]] = new_dup.iloc[:,[4,5,6,7]].values'''

### Check year and release_date discrepancies

In [80]:
'''year_dif = sr.loc[(~sr['release_date'].isnull())]
year_dif = year_dif.loc[year_dif['release_date'].str.len()>4]
year_dif = year_dif.loc[abs(year_dif['year']-year_dif['release_date'].str[:4].astype(int))>=1]

#year_dif.to_excel('new_spotify_year_dif.xlsx')

new_year_dif = pd.read_excel('year_dif.xlsx')
sr.iloc[new_year_dif['Unnamed: 0'].values, [3,4,5,6]] = new_year_dif.iloc[:,[4,5,6,7]].values'''

### Null

In [87]:
sr.loc[sr['track_id'].isnull()]
#sr.loc[sr['track_id'].isnull()].to_excel('new_spotify_correct_null.xlsx')

,title,artists,year,track_id,release_date,spotify.name,spotify.artists
0,(I Called Her) Tennessee,Tim Dugger,2012,NaN,NaN,NaN,NaN
4,110%,Jessie Ware,2012,NaN,NaN,NaN,NaN
22,4th And Vine,Sinéad O'Connor,2012,NaN,NaN,NaN,NaN
36,A Little Reminder That I'll Never Forget,Lostprophets,2012,NaN,NaN,NaN,NaN
42,A Prayer,Gordon Bahary,2012,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
19869,Skate,Silk Sonic,2021,NaN,NaN,NaN,NaN
19966,U&ME,Alt-J,2021,NaN,NaN,NaN,NaN
20001,What Other People Say,Demi Lovato,2021,NaN,NaN,NaN,NaN
20017,Working,Khalid,2021,NaN,NaN,NaN,NaN
